In [2]:
import pandas as pd
import numpy as np

import requests

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import os
import re

In [2]:
chromedriver = "/home/michael/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument('--dns-prefetch-disable')

# 1.0 Scraping Data
I need to scrape the dates and times for different sporting events in chicago from their respective sites

## 1.1 Bulls

In [6]:
def bulls_seasondata(startyear, years = 2):
    '''Function to pull all games for given year range for the bulls from baskeball reference'''
    columns = ['games','date','time','blank','blank2','location','opponent','result','timing','pts','oppts','w','l','streak','notes']
    finaldata = pd.DataFrame(columns = columns)
    driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
    
    for year in range(years):
        driver.get('https://www.basketball-reference.com/teams/CHI/{}_games.html'.format(startyear+year))
        time.sleep(3)
        seasondata = driver.find_element_by_xpath('//table[@id="games"]').get_attribute('outerHTML')
        newdata = pd.read_html(seasondata, header = 0)[0]
        newdata.columns = columns
        finaldata = finaldata.append(newdata)
        try:
            playoffdata = driver.find_element_by_xpath('//table[@id="games_playoffs"]').get_attribute('outerHTML')
            newdata = pd.read_html(playoffdata, header = 0)[0]
            newdata.columns = columns
            finaldata = finaldata.append(newdata)
        except:
            pass
    driver.close()
    return finaldata

In [7]:
bullsdata = bulls_seasondata(2014, years = 5)

In [8]:
bullsdata.head()

,games,date,time,blank,blank2,location,opponent,result,timing,pts,oppts,w,l,streak,notes
0,1,"Tue, Oct 29, 2013",8:00p ET,NaN,Box Score,@,Miami Heat,L,NaN,95,107,0,1,L 1,NaN
1,2,"Thu, Oct 31, 2013",8:00p ET,NaN,Box Score,NaN,New York Knicks,W,NaN,82,81,1,1,W 1,NaN
2,3,"Sat, Nov 2, 2013",7:30p ET,NaN,Box Score,@,Philadelphia 76ers,L,NaN,104,107,1,2,L 1,NaN
3,4,"Wed, Nov 6, 2013",7:00p ET,NaN,Box Score,@,Indiana Pacers,L,NaN,80,97,1,3,L 2,NaN
4,5,"Fri, Nov 8, 2013",8:00p ET,NaN,Box Score,NaN,Utah Jazz,W,NaN,97,73,2,3,W 1,NaN


In [51]:
bullsdata.to_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/bulls.pkl')

## 1.2 Cubs/Sox

In [21]:
def baseball_seasondata(startyear, years = 2):
    '''Function to pull all games for given year range for the cubs and sox from baseball reference'''
    basecolumns = ['games','date','blank2','team','location','opponent','result','runsfor','runsagainst','innings',
                   'record','rank','gb','win','loss','save','gametime','day/night','attend','streak','origsch','year']
    finaldata = pd.DataFrame(columns = basecolumns)
    driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
    teams = ['CHC','CHW']
    
    for team in teams:
        for year in range(years):
            driver.get('https://www.baseball-reference.com/teams/{}/{}-schedule-scores.shtml'.format(team, startyear+year))
            time.sleep(3)
            seasondata = driver.find_element_by_xpath('//table[@id="team_schedule"]').get_attribute('outerHTML')
            newdata = pd.read_html(seasondata, header = 0)[0]
            newdata['year'] = startyear + year
            newdata.columns = basecolumns
            finaldata = finaldata.append(newdata)
    driver.close()
    return finaldata

In [19]:
newdata.head()

,Gm#,Date,Unnamed: 2,Tm,Unnamed: 4,Opp,W/L,R,RA,Inn,...,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled
0,1,"Monday, Apr 1",boxscore,CHC,@,PIT,W,3,1,NaN,...,1,Tied,Samardzija,Burnett,Fujikawa,2:59,D,39078,+,NaN
1,2,"Wednesday, Apr 3",boxscore,CHC,@,PIT,L,0,3,NaN,...,1,Tied,Rodriguez,Jackson,Grilli,2:55,N,27667,-,NaN
2,3,"Thursday, Apr 4",boxscore,CHC,@,PIT,W,3,2,NaN,...,1,Tied,Wood,McDonald,Marmol,2:41,D,11634,+,NaN
3,4,"Friday, Apr 5",boxscore,CHC,@,ATL,L,1,4,NaN,...,2,1.0,Minor,Feldman,Kimbrel,2:36,N,33443,-,NaN
4,5,"Saturday, Apr 6",boxscore,CHC,@,ATL,L-wo,5,6,NaN,...,2,1.0,O'Flaherty,Marmol,NaN,3:12,N,38498,--,NaN


In [22]:
baseballdata = baseball_seasondata(2013, years = 5)

In [24]:
baseballdata.tail()

,games,date,blank2,team,location,opponent,result,runsfor,runsagainst,innings,...,gb,win,loss,save,gametime,day/night,attend,streak,origsch,year
164,160,"Friday, Sep 29",boxscore,CHW,@,CLE,L,1,10,NaN,...,35.0,Bauer,Pelfrey,NaN,2:57,N,26983,-,NaN,2017
165,161,"Saturday, Sep 30",boxscore,CHW,@,CLE,W,2,1,NaN,...,34.0,Fulmer,Clevinger,Minaya,3:13,N,33173,+,NaN,2017
166,Gm#,October,NaN,Tm,NaN,Opp,W/L,R,RA,Inn,...,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled,2017
167,162,"Sunday, Oct 1",boxscore,CHW,@,CLE,L,1,3,NaN,...,35.0,Tomlin,Volstad,Allen,2:37,D,30036,-,NaN,2017
168,Gm#,Date,NaN,Tm,NaN,Opp,W/L,R,RA,Inn,...,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled,2017


In [25]:
baseballdata.to_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/baseball.pkl')

## 1.3 Bears

In [3]:
def bears_seasondata(startyear, years = 2):
    '''Function to pull all games for given year range for the bears from football reference'''
    bearcol = ['games','day','date','time','blank2','result','ot','record','location','opponent','pts', 'oppts',
               '1stD', 'TotYd', 'PassY', 'RushY', 'TO', '1stD.1', 'TotYd.1', 'PassY.1', 'RushY.1', 'TO.1', 'Offense', 'Defense', 'Sp. Tms', 'year']
    finaldata = pd.DataFrame(columns = bearcol)
    driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
        
    for year in range(years):
        driver.get('https://www.pro-football-reference.com/teams/chi/{}_games.htm'.format(startyear+year))
        time.sleep(3)
        seasondata = driver.find_element_by_xpath('//table[@id="games"]').get_attribute('outerHTML')
        newdata = pd.read_html(seasondata, header = 0)[0]
        newdata['year'] = startyear + year
        newdata.columns = bearcol
        finaldata = finaldata.append(newdata)
    driver.close()
    return finaldata

In [4]:
bearsdata = bears_seasondata(2013, years = 5)

In [5]:
bearsdata.head()

,games,day,date,time,blank2,result,ot,record,location,opponent,...,TO,1stD.1,TotYd.1,PassY.1,RushY.1,TO.1,Offense,Defense,Sp. Tms,year
0,Week,Day,Date,NaN,NaN,NaN,OT,Rec,NaN,Opp,...,TO,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Sp. Tms,2013
1,1,Sun,September 8,1:02PM ET,boxscore,W,NaN,1-0,NaN,Cincinnati Bengals,...,1,18,340,277,63,3,-0.93,-2.65,3.28,2013
2,2,Sun,September 15,1:03PM ET,boxscore,W,NaN,2-0,NaN,Minnesota Vikings,...,4,19,350,227,123,3,-0.92,3.26,1.56,2013
3,3,Sun,September 22,8:30PM ET,boxscore,W,NaN,3-0,@,Pittsburgh Steelers,...,NaN,21,459,379,80,5,1.41,15.78,0.13,2013
4,4,Sun,September 29,1:03PM ET,boxscore,L,NaN,3-1,@,Detroit Lions,...,4,23,387,228,159,3,-2.72,1.64,-6.73,2013


In [6]:
bearsdata.shape

(90, 26)

In [7]:
bearsdata.to_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/bears.pkl')

## 1.4 Blackhawks

In [3]:
def hawks_seasondata(startyear, years = 2):
    '''Function to pull all games for given year range for the blackhawks from hockey reference'''
    gamecolumns = ['games','date','time','locations','opponent','pts','oppts','result','shootout','wins','losses','sol','streak',
                   'blank','S', 'PIM', 'PPG', 'SHG', 'blank3', 'S.1', 'PIM.1', 'PPG.1','SHG.1', 'Notes']
    playoffcolumns = ['games','date','time','locations','opponent','pts','oppts','result','ot','wins','losses','streak',
                      'blank','S', 'PIM', 'PPG', 'SHG', 'blank3', 'S.1', 'PIM.1', 'PPG.1','SHG.1', 'Notes']
    finaldata = pd.DataFrame()#columns = gamecolumns)
    driver = webdriver.Chrome(chromedriver,chrome_options=chrome_options)
    
    for year in range(years):
        driver.get('https://www.hockey-reference.com/teams/CHI/{}_games.html'.format(startyear+year))
        time.sleep(3)
        seasondata = driver.find_element_by_xpath('//table[@id="games"]').get_attribute('outerHTML')
        newdata = pd.read_html(seasondata, header = 1)[0]
        #newdata.columns = gamecolumns
        finaldata = finaldata.append(newdata)
        try:
            playoffdata = driver.find_element_by_xpath('//table[@id="games_playoffs"]').get_attribute('outerHTML')
            newdata = pd.read_html(playoffdata, header = 1)[0]
            #newdata.columns = playoffcolumns
            finaldata = finaldata.append(newdata)
        except:
            pass
    driver.close()
    return finaldata

In [4]:
hawksdata = hawks_seasondata(2014, years = 5)

In [5]:
hawksdata.head()

,Att.,Date,GA,GF,GP,L,LOG,Notes,OL,Opponent,...,Time,Unnamed: 12,Unnamed: 13,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 3,Unnamed: 7,Unnamed: 8,W
0,NaN,2013-10-01,4,6,1,0,NaN,NaN,0,Washington Capitals,...,8:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,W,NaN,1
1,NaN,2013-10-05,3,2,2,0,NaN,NaN,1,Tampa Bay Lightning,...,8:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,L,SO,1
2,NaN,2013-10-09,3,2,3,1,NaN,NaN,1,St. Louis Blues,...,8:00 PM,NaN,NaN,NaN,NaN,NaN,@,L,NaN,1
3,NaN,2013-10-11,2,3,4,1,NaN,NaN,1,New York Islanders,...,8:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,W,NaN,2
4,NaN,2013-10-12,1,2,5,1,NaN,NaN,1,Buffalo Sabres,...,8:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,W,NaN,3


In [6]:
hawksdata.to_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/hawks.pkl')

## 1.5 Scraped Data Cleanup

In [3]:
bullsdata = pd.read_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/bulls.pkl')
bullsdata = bullsdata[bullsdata['location'] != '@']
bullsdata = bullsdata[bullsdata['games'] != 'G']

bullsdata['datetimeraw'] = bullsdata['date'] +' '+ bullsdata['time']
bullsdata['datetime'] = pd.to_datetime(bullsdata['datetimeraw']) + pd.DateOffset(hours = -1)

bullsdata = bullsdata[['games','datetime']]
bullsdata['location'] = 'unitedcenter'
bullsdata['team'] = 'bulls'
bullsdata.head()

,games,datetime,location,team
1,2,2013-10-31 19:00:00,unitedcenter,bulls
4,5,2013-11-08 19:00:00,unitedcenter,bulls
5,6,2013-11-11 19:00:00,unitedcenter,bulls
7,8,2013-11-16 19:00:00,unitedcenter,bulls
8,9,2013-11-18 19:00:00,unitedcenter,bulls


In [4]:
baseballdata = pd.read_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/baseball.pkl')
baseballdata = baseballdata[baseballdata['location'] != '@']
baseballdata = baseballdata[baseballdata['games'] != 'Gm#']
baseballdata['time'] = np.where(baseballdata['day/night'] == 'D', '1:00p', '7:00p')
baseballdata['year'] = [str(x) for x in baseballdata['year']]
baseballdata['date'] = [x.split(',')[1] for x in baseballdata['date']]
baseballdata['date'] = [x.split('(')[0] for x in baseballdata['date']]
baseballdata['datetimeraw'] = baseballdata['date'] + ', ' + baseballdata['year']+ ' ' + baseballdata['time']
baseballdata['datetime'] = pd.to_datetime(baseballdata['datetimeraw'])
baseballdata['location'] = np.where(baseballdata['team'] == 'CHC', 'wrigley', 'uscellular')
baseballdata['team'] = np.where(baseballdata['team'] == 'CHC', 'cubs', 'sox')
baseballdata = baseballdata[['games','datetime','location','team','attend']]
baseballdata.head(5)

,games,datetime,location,team,attend
6,7,2013-04-08 13:00:00,wrigley,cubs,40083
7,8,2013-04-09 19:00:00,wrigley,cubs,30065
8,9,2013-04-11 13:00:00,wrigley,cubs,25460
9,10,2013-04-12 13:00:00,wrigley,cubs,30996
10,11,2013-04-13 13:00:00,wrigley,cubs,34778


In [5]:
bearsdata = pd.read_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/bears.pkl')
bearsdata = bearsdata[bearsdata['location'] != '@']
bearsdata = bearsdata[bearsdata['games'] != 'Week']
bearsdata = bearsdata[bearsdata['opponent'] != 'Bye Week']
bearsdata['year'] = [str(x) for x in bearsdata['year']]
bearsdata['datetimeraw'] = bearsdata['date'] + ', ' + bearsdata['year']+ ' ' + bearsdata['time']
bearsdata['datetime'] = pd.to_datetime(bearsdata['datetimeraw'])
bearsdata['datetime'] = bearsdata['datetime'].dt.round('H') + pd.DateOffset(hours = -1)
bearsdata = bearsdata[['games','datetime']]
bearsdata['location'] = 'soldierfield'
bearsdata['team'] = 'bears'
bearsdata.head()

,games,datetime,location,team
1,1,2013-09-08 12:00:00,soldierfield,bears
2,2,2013-09-15 12:00:00,soldierfield,bears
5,5,2013-10-06 12:00:00,soldierfield,bears
6,6,2013-10-10 19:00:00,soldierfield,bears
10,10,2013-11-10 12:00:00,soldierfield,bears


In [6]:
hawksdata = pd.read_pickle('/home/michael/Documents/Projects/divvydataproject/data/SportballGames/hawks.pkl')
hawksdata = hawksdata[hawksdata['Unnamed: 3'] != '@']
hawksdata = hawksdata[hawksdata['Date'] != 'Date']
hawksdata['datetimeraw'] = hawksdata['Date'] + ' ' + hawksdata['Time']
hawksdata['datetime'] = pd.to_datetime(hawksdata['datetimeraw'])
hawksdata['datetime'] = hawksdata['datetime'].dt.round('H')
hawksdata = hawksdata[['GP','datetime','Att.']]
hawksdata.columns = ['games','datetime','attend']
hawksdata['location'] = 'unitedcenter'
hawksdata['team'] = 'blackhawks'
hawksdata.head()

,games,datetime,attend,location,team
0,1,2013-10-01 20:00:00,NaN,unitedcenter,blackhawks
1,2,2013-10-05 20:00:00,NaN,unitedcenter,blackhawks
3,4,2013-10-11 20:00:00,NaN,unitedcenter,blackhawks
4,5,2013-10-12 20:00:00,NaN,unitedcenter,blackhawks
6,7,2013-10-17 20:00:00,NaN,unitedcenter,blackhawks


In [7]:
sportsdata = pd.concat([bullsdata,baseballdata,bearsdata,hawksdata])

In [8]:
locations = {'unitedcenter' : (41.8749965, -87.671163982),
            'wrigley' : (41.948437, -87.655334),
            'soldierfield' : (41.85749657, -87.6166642),
            'uscellular': (41.824663368, -87.633664132)}

In [10]:
sportsdata['gpslocation'] = sportsdata['location'].map(locations)

In [12]:
sportsdata.head()

,attend,datetime,games,location,team,gpslocation
1,NaN,2013-10-31 19:00:00,2,unitedcenter,bulls,"(41.8749965, -87.671163982)"
4,NaN,2013-11-08 19:00:00,5,unitedcenter,bulls,"(41.8749965, -87.671163982)"
5,NaN,2013-11-11 19:00:00,6,unitedcenter,bulls,"(41.8749965, -87.671163982)"
7,NaN,2013-11-16 19:00:00,8,unitedcenter,bulls,"(41.8749965, -87.671163982)"
8,NaN,2013-11-18 19:00:00,9,unitedcenter,bulls,"(41.8749965, -87.671163982)"


In [13]:
sportsdata.groupby('gpslocation')[['games']].count()

,games
gpslocation,
"(41.824663368, -87.633664132)",405
"(41.85749657, -87.6166642)",40
"(41.8749965, -87.671163982)",448
"(41.948437, -87.655334)",405


In [14]:
sportsdata.to_pickle('/home/michael/Documents/Projects/divvydataproject/data/sporteventdata.pkl')